In [1]:
import pandas as pd
from collections import Counter 
from helper import load_data,pipeline, unique_drugs, generate_drugs_df
import re

%pylab inline 
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
df_temp = load_data()
#c = unique_drugs(trials)
#df_data, exceptions = generate_drugs_df(c, trials)
df_temp = df_temp.reset_index()


## expanded unique drugs to get a dataframe we can group by
dupes = []
for idx, item_list in enumerate(df_temp["Primary Drugs"]):
    split_list = item_list.split(",")
    if len(split_list)>1:
        for i in split_list:
            df_temp.append(df_temp.iloc[idx,:])
            df_temp.iloc[-1,-3]=i
        dupes.append(idx)

df_trials = df_temp.drop(dupes)


In [5]:
df_trials.shape

(1812, 44)

In [4]:
drug_list = df_trials["Primary Drugs"].unique()
phase_of_trial_list = df_trials["Phase of Trial"].unique()
df_num_of_pariticpants = pd.DataFrame(np.zeros([len(drug_list),len(phase_of_trial_list)]),columns=phase_of_trial_list)
df_num_of_pariticpants['drug'] = drug_list
df_num_of_pariticpants = df_num_of_pariticpants.set_index('drug')
grouped_phases = df_trials.groupby(["Primary Drugs","Phase of Trial"])["Planned Subject Number"].agg('mean')
cleaner_groups = grouped_phases.reset_index()

for drug in drug_list:
    for item in cleaner_groups[cleaner_groups["Primary Drugs"]==drug].iterrows():   
        phase = list(item)[1]['Phase of Trial']
        count = list(item)[1]['Planned Subject Number']
        df_num_of_pariticpants.loc[drug,phase] = count


In [5]:
df_drugs = pd.read_excel('bcdrugs.xlsx')
df_drugs['drug'] = df_drugs.apply(lambda x: x["Drug Name"].split("-")[0], axis=1)
df_num_of_pariticpants_red = df_num_of_pariticpants.drop('Phase 0',axis=1)
df_num_of_pariticpants_red = df_num_of_pariticpants_red[(df_num_of_pariticpants_red['Phase I']!=0) | (df_num_of_pariticpants_red['Phase II']!=0) | (df_num_of_pariticpants_red['Phase I/II']!=0)]

In [6]:
# Create boolean to determine if the current passed. Based on whether there were participants from the next phase
#phases = [item for item in df_num_of_pariticpants_red.columns if 'I' in item]
df_num_of_pariticpants_red['Phase I Pass'] = df_num_of_pariticpants_red['Phase II']!=0
df_num_of_pariticpants_red['Phase II Pass'] = df_num_of_pariticpants_red['Phase III']!=0
df_num_of_pariticpants_red['Phase I/II Pass'] = df_num_of_pariticpants_red['Phase III']!=0
df_num_of_pariticpants_red['Phase III Pass'] = df_num_of_pariticpants_red['Phase IV']!=0
df_num_of_pariticpants_red['Phase II/III Pass'] = df_num_of_pariticpants_red['Phase IV']!=0

In [7]:
df_data = df_num_of_pariticpants_red

Getting number of organizations for each drug

In [8]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [9]:
def remove_org_dupes(x):
    drop_list = []
    #print(type(x))
    if type(x) != type('astring'):
        return []
    if x == np.nan:
        return []
    split_orgs = x.split(",")
    if len(split_orgs)>1:
        for idx1, i in enumerate(split_orgs[:-1]):
            for idx2, j in enumerate(split_orgs[idx1+1:]):
                #print(i,idx1, j ,idx2, fuzz.ratio(i,j))
                if fuzz.ratio(i,j) > 50:
                    drop_list.append(idx1+idx2+1)
        #print(f'Drop list: {drop_list}')
        #print(f'FINAL: {split_orgs[list(set(drop_list))]}')
        #split_orgs[not set(drop_list)F
        #print (list(set(drop_list)))
        output = np.delete(split_orgs, list(set(drop_list)))
        return [item.strip() for item in output]
    return [item.strip() for item in [x]]

In [10]:
#trials["Organisations"] = trials["Organisations"].apply(str)
df_trials['clean Organisations'] = df_trials['Organisations'].apply(remove_org_dupes)

In [11]:
df_trials['clean Organisations'] = df_trials['Organisations'].apply(remove_org_dupes)
df_trials['number of clean Organisations'] = df_trials['clean Organisations'].apply(lambda x:len(x))

In [12]:
df_org_count = df_trials[['Primary Drugs','number of clean Organisations']]
df_org_count["Primary Drugs"] = df_org_count["Primary Drugs"].apply(lambda x: x.strip())

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
grouped_orgs = df_org_count.groupby("Primary Drugs")["number of clean Organisations"].agg('max')
grouped_orgs = grouped_orgs.reset_index()

In [14]:
grouped_orgs.head()

,Primary Drugs,number of clean Organisations
0,212 Pb TCMC trastuzumab,2
1,852A,1
2,ABBV 176,1
3,ABT 751,1
4,ABT 767,2


In [15]:
df_data = df_data.merge(grouped_orgs, left_index=True, right_on="Primary Drugs", how='left')
df_data.set_index('Primary Drugs', inplace=True)

In [16]:
def trial_history_to_table(th):
    rows = str.split(th, ';')
    table = [str.split(row, '|') for row in rows]
    return pd.DataFrame(table,columns=['Date','Status','Notes'])

trial_history_to_table(df_temp["Trial History"].values[15])

,Date,Status,Notes
0,2019-02-27,Results,"Results (n=62) assessing safety, tolerability,..."
1,2019-02-12,Other trial event,Last checked against the ClinicalTrials.gov re...
2,2019-02-01,Status change - completed,"Status changed from active, no longer recruiti..."
3,2018-06-05,Results,Results of pharmacokinetic analsysis (n=58) pr...
4,2018-06-05,Results,Results presented at the 54th Annual Meeting o...
5,2018-06-03,Results,Results published in the Media Release
6,2018-05-30,Completion date,Planned End Date changed from 1 Aug 2017 to 1 ...
7,2018-05-30,Other trial event,Planned primary completion date changed from 1...
8,2018-05-30,"Status change - active, no longer recruiting","Status changed from recruiting to active, no l..."
9,2018-05-16,Other trial event,"According to a Forty Seven media release, the ..."


In [17]:
#df_original.groupby()
df_trials["time_diff"] = df_trials["trial_end"] - df_trials["trial_start"]

In [18]:
df_trials["Trial Initiation date"]

0       22 Jun 2010 (actual)
1       01 Dec 2005 (actual)
2       24 Aug 2011 (actual)
3       17 Jan 2012 (actual)
4       01 Jan 2014 (actual)
5       21 Feb 2014 (actual)
7       11 Apr 2007 (actual)
9       21 Nov 2017 (actual)
10      01 Jun 2011 (actual)
11      14 Jan 2017 (actual)
12      02 Apr 2013 (actual)
13      05 Mar 2018 (actual)
14      14 Mar 2017 (actual)
15      01 Aug 2014 (actual)
16      11 Mar 2010 (actual)
17      01 Sep 2007 (actual)
18      28 Oct 2002 (actual)
19      06 Nov 2015 (actual)
20      18 Jan 2001 (actual)
21      01 Sep 2005 (actual)
25      01 Jul 1999 (actual)
26      11 Dec 2009 (actual)
27      12 Aug 2015 (actual)
29      01 Sep 2004 (actual)
30      01 May 2011 (actual)
31      01 Nov 1999 (actual)
32      01 Apr 2010 (actual)
33      01 Aug 2001 (actual)
34      01 May 2010 (actual)
35      13 Dec 2013 (actual)
                ...         
1838    01 Aug 2009 (actual)
1839    01 Jul 2009 (actual)
1840    01 Oct 2002 (actual)
1841    01 Dec

Note: Checked that trial start and end date and time difference values are accurate. Need to get subset.

In [19]:
df_trials[["Trial ID","Trial Initiation date","Trial End date","trial_start","trial_end","time_diff","Primary Drugs"]].sort_values("time_diff",ascending=True).head()

,Trial ID,Trial Initiation date,Trial End date,trial_start,trial_end,time_diff,Primary Drugs
1752,700037695,01 Oct 2008 (actual),01 Oct 2008 (actual),2008-10-01 00:00:00,2008-10-01 00:00:00,0 days,Neratinib
1838,700045997,01 Aug 2009 (actual),01 Aug 2009 (actual),2009-08-01 00:00:00,2009-08-01 00:00:00,0 days,Axitinib
1857,700030416,01 Jan 2008 (actual),01 Jan 2008 (actual),2008-01-01 00:00:00,2008-01-01 00:00:00,0 days,BMS 690514
272,700291322,23 Nov 2017 (actual),14 Dec 2017 (actual),2017-11-23 00:00:00,2017-12-14 00:00:00,21 days,Varlitinib
1816,700054377,01 Feb 2010 (actual),01 Mar 2010 (actual),2010-02-01 00:00:00,2010-03-01 00:00:00,28 days,Bosutinib


In [20]:
#removes trials that have zero time delta
zero_time_delta_mask = df_trials["time_diff"] > np.timedelta64(0)
df_trials = df_trials[zero_time_delta_mask]

In [21]:
drug_list = df_trials["Primary Drugs"].unique()
phase_of_trial_list = df_trials["Phase of Trial"].unique()
df_time_diff = pd.DataFrame(np.zeros([len(drug_list),len(phase_of_trial_list)]),columns=phase_of_trial_list)
df_time_diff['Primary Drugs'] = drug_list
df_time_diff = df_time_diff.set_index('Primary Drugs')

df_trials["nano_time_diff"] = df_trials["time_diff"].astype(np.int64)
grouped_time_deltas = pd.to_timedelta(df_trials.groupby(["Primary Drugs","Phase of Trial"])["nano_time_diff"].agg('mean')).reset_index()

for drug in drug_list:
    for item in grouped_time_deltas[grouped_time_deltas["Primary Drugs"]==drug].iterrows():   
        phase = list(item)[1]['Phase of Trial']
        count = list(item)[1]['nano_time_diff']
        df_time_diff.loc[drug,phase] = count
    
df_time_diff.columns=[i+' trial length' for i in df_time_diff.columns]

In [28]:
df_data.merge(df_time_diff, left_index=True, right_index=True, how='right').shape


(432, 21)

In [27]:
df_data.shape

(406, 13)

In [25]:
df_time_diff.shape

(432, 8)